In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Week 09 Handson
In this week handson, we'll try to play with Kaggle, which is one of the biggest data science community platforms. We will try to join a Kaggle competition by building a model that can predict MoA (Mechanisms of Action) in drugs development. General guide about what you need to do:
1. Register to Kaggle (if you haven't had an account yet) with your full name,
2. Download the dataset,
3. Build a model,
4. Perform an inference to the given testing data,
5. Submit the inference result

Competition link: [cick here](https://www.kaggle.com/c/lish-moa/overview)

Submission:
1. This jupyter notebook: there are at least three blocks of codes, which are data preparation, modelling and inference. However, you are free to modify, e.g., further breaking down the data prepration block to EDA and data preprocessing, etc.
2. Csv file that is submitted to the competition.
3. Screenshot of your posisition in the leaderboard (jpg file).

Zip those three files above, with a file name of "W09_your-student-id_your-name.zip" and submit to the course portal. In case the allowable size is exceeded, you can upload to, e.g., gdrive first, then upload a txt file containing that download url to the course portal. In such case, please make sure that the url is publicly open.

In [ ]:
df_train = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
display(df_train.head(3))
print('train data size', df_train.shape)

df_target_ns = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
display(df_target_ns.head(3))
print('train target nonscored size', df_target_ns.shape)


df_target_s = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
display(df_target_s.head(3))
print('train target scored size', df_target_s.shape)


df_test = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
display(df_test.head(3))
print('test data size', df_test.shape)

df_sample = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')
display(df_sample.head(3))
print('sample submission size', df_sample.shape)

# Data Preparation

There is a better EDA that has been done by the community in the following links: <br> -https://www.kaggle.com/sinamhd9/mechanisms-of-action-moa-tutorial <br> - https://www.kaggle.com/gunesevitan/mechanisms-of-action-moa-prediction-eda

From the EDA it can be inferred that there are rows which are called controlled groups and always results in 0. I will always predict 0 for cp_type == cntrl_vehichle. 

In [ ]:
# cp_time and cp_dose are two categorical data and will be preprocessed as follows
def preprocess(df):
    df['cp_time'] = df['cp_time'].map({24:1, 48:2, 72:3})
    df['cp_dose'] = df['cp_dose'].map({'D1':0, 'D2':1})
    return df

df_train = preprocess(df_train)
df_test = preprocess(df_test)

# Modelling

In [ ]:
import sklearn
from xgboost import XGBClassifier
from sklearn.model_selection import KFold
from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import log_loss
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
NFOLDS = 5

In [ ]:
# Dropping ID column
X = df_train.iloc[:,1:].to_numpy()
X_test = df_test.iloc[:,1:].to_numpy()
y = df_target_s.iloc[:,1:].to_numpy() 

In [ ]:
classifier = MultiOutputClassifier(XGBClassifier(tree_method='gpu_hist'))

clf = Pipeline([('encode', CountEncoder(cols=[0, 2])),
                ('classify', classifier)
               ])

In [ ]:
params = {'classify__estimator__colsample_bytree': 0.6522,
          'classify__estimator__gamma': 3.6975,
          'classify__estimator__learning_rate': 0.0503,
          'classify__estimator__max_delta_step': 2.0706,
          'classify__estimator__max_depth': 10,
          'classify__estimator__min_child_weight': 31.5800,
          'classify__estimator__n_estimators': 166,
          'classify__estimator__subsample': 0.8639
         }

_ = clf.set_params(**params)

In [ ]:
oof_preds = np.zeros(y.shape)
test_preds = np.zeros((df_test.shape[0], y.shape[1]))
oof_losses = []
kf = KFold(n_splits=NFOLDS)
for fn, (trn_idx, val_idx) in enumerate(kf.split(X, y)):
    print('Starting fold: ', fn)
    X_train, X_val = X[trn_idx], X[val_idx]
    y_train, y_val = y[trn_idx], y[val_idx]
    
    # drop where cp_type==ctl_vehicle (baseline)
    ctl_mask = X_train[:,0]=='ctl_vehicle'
    X_train = X_train[~ctl_mask,:]
    y_train = y_train[~ctl_mask]
    
    clf.fit(X_train, y_train)
    val_preds = clf.predict_proba(X_val) # list of preds per class
    val_preds = np.array(val_preds)[:,:,1].T # take the positive class
    oof_preds[val_idx] = val_preds
    
    loss = log_loss(np.ravel(y_val), np.ravel(val_preds))
    oof_losses.append(loss)
    preds = clf.predict_proba(X_test)
    preds = np.array(preds)[:,:,1].T # take the positive class
    test_preds += preds / NFOLDS
    
print(oof_losses)
print('Mean OOF loss across folds', np.mean(oof_losses))
print('STD OOF loss across folds', np.std(oof_losses))

In [ ]:
control_mask = df_train['cp_type']=='ctl_vehicle'
oof_preds[control_mask] = 0

print('OOF log loss: ', log_loss(np.ravel(y), np.ravel(oof_preds)))

# Inference

In [ ]:
control_mask = df_test['cp_type']=='ctl_vehicle'

test_preds[control_mask] = 0

In [ ]:
df_sample.iloc[:,1:] = test_preds
df_sample.to_csv('submission.csv', index=False)